### GSE104897 re-analysis (schattling et al. - bassoon paper)

In [1]:
library(GEOquery)
library(tidyverse)

gse = getGEO('GSE104897')
samples = gse[[1]][[1]]

getGEOSuppFiles('GSE104897', baseDir = 'P:/pro_mitoproteomics_de/repos/')

untar('P:/pro_mitoproteomics_de/repos/GSE104897/GSE104897_RAW.tar', exdir = 'P:/pro_mitoproteomics_de/repos/GSE104897/rawfiles')

files = list.files('P:/pro_mitoproteomics_de/repos/GSE104897/rawfiles')
files = files[grepl("raw", files)]

df = data.frame()

df = data.frame(gene_id = numeric(0), expr = numeric(0));

colnames(df) = c('gene', 'expr')

for (file in files) {
    dir = paste0('P:/pro_mitoproteomics_de/repos/GSE104897/rawfiles/', file)
    x = read.table(dir)
    colnames(x) = c('gene', paste0('expr_', file))
    df = merge(df, x, by = 'gene', all = T)
}

colnames(df)[-c(1,2)] = samples

head(df)

library('org.Mm.eg.db') # remember to install it if you don't have it already
symbols <- mapIds(org.Mm.eg.db, keys = df$gene, keytype = "ENSEMBL", column="SYMBOL")

df$gene = symbols %>% unlist() %>% as.vector()
df = df[!is.na(df$gen), ]
df = df[!duplicated(df$gene), ]

df = df[-c(2)]

rownames(df) = 1:nrow(df)

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; vie

,size,isdir,mode,mtime,ctime,atime,exe
,<dbl>,<lgl>,<octmode>,<dttm>,<dttm>,<dttm>,<chr>
P:/pro_mitoproteomics_de/repos//GSE104897/GSE104897_RAW.tar,7342080,FALSE,666,2020-11-09 16:38:05,2020-10-01 11:33:47,2020-11-09 16:38:05,no


,gene,expr,SC-Healthy-rep1,SC-Healthy-rep2,SC-Healthy-rep3,SC-Healthy-rep4,SC-Healthy-rep5,SC-EAE-rep1,SC-EAE-rep2,SC-EAE-rep3,...,MN-Healthy-rep1,MN-Healthy-rep2,MN-Healthy-rep3,MN-Healthy-rep4,MN-Healthy-rep5,MN-EAE-rep1,MN-EAE-rep2,MN-EAE-rep3,MN-EAE-rep4,MN-EAE-rep5
,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSMUSG00000000001,NA,437,171,615,1229,878,819,250,755,...,227,152,184,424,1444,102,0,32,30,288
2,ENSMUSG00000000003,NA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSMUSG00000000028,NA,33,22,67,92,81,75,34,54,...,59,0,86,84,10,34,15,3,0,0
4,ENSMUSG00000000031,NA,5,3,9,2,2,33,29,31,...,0,0,0,1,0,9,0,5,0,104
5,ENSMUSG00000000037,NA,8,5,15,9,27,8,8,6,...,3,0,0,21,0,7,0,0,0,0
6,ENSMUSG00000000049,NA,8,1,7,6,11,4,2,3,...,0,0,15,21,0,0,0,0,0,0


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: IRanges

Loading required package: S4Vectors


Attaching package: 'S4Vectors'


The following objects are masked from 'package:dplyr':

    first, rename


The following object is masked from 'package:tidyr':

    expand


The following object is masked from 'package:base':

    expand.grid



Attaching package: 'IRanges'


The following objects are masked from 'package:dplyr':

    collapse, desc, slice


The following object is masked from 'package:purrr':

    reduce


The following object is masked from 'package:grDevices':

    windows



Attaching package: 'AnnotationDbi'


The following object is masked from 'package:dplyr':

    select




'select()' returned 1:many mapping between keys and columns



In [2]:
# https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#countmat
library(DESeq2)
genes = df$gene
df = df[, grepl('MN-', colnames(df))]
rownames(df) = genes
condition = factor(c(rep('ctl', 5), rep('eae', 5)))
row.names(df) = genes
head(df)

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: DelayedArray

Loading required package: matrixStats


Attaching package: 'matrixStats'


The following object is masked from 'package:dplyr':

    count


The following objects are masked from 'package:Biobase':

    anyMissing, rowMedians



Attaching package: 'DelayedArray'


The following objects are masked from 'package:matrixStats':

    colMaxs, colMins, colRanges, rowMaxs, rowMins, rowRanges


The following object is masked from 'package:purrr':

    simplify


The following objects are masked from 'package:base':

    aperm, apply, rowsum




,MN-Healthy-rep1,MN-Healthy-rep2,MN-Healthy-rep3,MN-Healthy-rep4,MN-Healthy-rep5,MN-EAE-rep1,MN-EAE-rep2,MN-EAE-rep3,MN-EAE-rep4,MN-EAE-rep5
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Gnai3,227,152,184,424,1444,102,0,32,30,288
Pbsn,0,0,0,0,0,0,0,0,0,0
Cdc45,59,0,86,84,10,34,15,3,0,0
H19,0,0,0,1,0,9,0,5,0,104
Scml2,3,0,0,21,0,7,0,0,0,0
Apoh,0,0,15,21,0,0,0,0,0,0


In [3]:
coldata = as.matrix(condition)
row.names(coldata) = names(df)

In [4]:
colnames(coldata) = 'condition'

In [5]:
cds = DESeqDataSetFromMatrix(df, coldata, ~condition)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"


In [6]:
cds

class: DESeqDataSet 
dim: 25629 10 
metadata(1): version
assays(1): counts
rownames(25629): Gnai3 Pbsn ... Rhbg LOC108168586
rowData names(0):
colnames(10): MN-Healthy-rep1 MN-Healthy-rep2 ... MN-EAE-rep4
  MN-EAE-rep5
colData names(1): condition

In [7]:
keep <- rowSums(counts(cds)) >= 10
cds <- cds[keep,]

In [8]:
cds$condition <- factor(cds$condition, levels = c("ctl", "eae"))

In [9]:
cds$condition <- relevel(cds$condition, ref = "ctl")

In [10]:
cds$condition <- droplevels(cds$condition)

In [11]:
cds = DESeq(cds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [12]:
res <- results(cds)

In [13]:
write.table(res, 'P:/pro_mitoproteomics_de/repos/GSE104897/bsn_rnaseq.txt', sep = '\t', row.names = T)

In [ ]:
upreg = c('Apol6', 'Col5a3', 'Sh3tc1', 'Bsn', 'Arl4d', 'Vgf', 'Elmo2', 'Cyb5r2', 'Igsf8', 'Nucb1', 'Dpf2', 'Pttg1', 'Clip2', 'Dync1h1', 'Tle3',
'Ankrd54', 'Kif1a', 'Map1a', 'Gm37829', 'Tbc1d10b')

df[upreg, ]

In [ ]:
counts(cds)